# Welcome to Lung Cancer Center of Excellence Workshop 2023!

**This is data description and code snippets made for R users, if you prefer Python, please click here**

Dev questrions:
The mutations are not annotated with genes - where do we get that info?

make all ids either patient id or tumor id

## Table of content:
* [General information](#General-information)
* [How to download the data](#How-to_download-the-data)
* [Required software and packages](#Required-software-and-packages)
* [I am not able to install required software/packages on my computer, what should I do?](#I-am-not-able-to-install)

Sections containing data description:
* [Somatic mutation data](#Somatic-mutation-data)
    * [Reading data into R](#Somatic-Reading)
    * [Column description](#Somatic-Columns)
    * [How to extract mutation coordinates](#Somatic-Coordinates)

## General information <a class="anchor" id="General-information"></a>

The workshop is based on data generated by the TRACERx (Tracking Cancer Evolution through Therapy (Rx) project. It is a large-scale research project aimed at understanding how cancer evolves and spreads over time. The project involves collecting multiple samples of tumours and analysing them in detail to identify genetic changes that occur as the cancer progresses.

Overall, 1,644 tumour regions were sampled at surgery or during follow-up from the first 421 patients with non-small cell lung cancer (NSCLC). The samples were subjected to whole exome sequencing (WES) and RNA-seq and thoroughly analysed. The results obtained from this analysis serve as the foundation for generating hypotheses during this workshop.

The available data include:
1. patient pathology and clinical data
2. somatic mutations
3. somatic copy number alterations
4. expression data

While rudimentary plots summarising the data are available below, we highly encourage you to have a brief look through the [collection of TRACERx papers](https://www.nature.com/collections/haffgaicaf). In particular, somatic point and copy number alterations are discussed [here](https://www.nature.com/articles/s41586-023-05783-5) and expression changes are examined [here](https://www.nature.com/articles/s41586-023-05706-4).

## How to download the data <a class="anchor" id="How-to_download-the-data"></a>
Please head to [zenodo](https://zenodo.org/record/7822002) and press download:

![zenodo](support_illustrations/zenodo.png)

The archive `figurecode.zip` will be downloaded, please unzip it. It will produce the folder `figurecode`. While the folder contains a multitude of files, we believe a handful of them can go a long way in terms of hypothesis generation:

* 20221109_TRACERx421_all_tumour_df.rds
* [Somatic single point mutations and small indels](#Somatic-mutation-data): 20221123_TRACERx421_mutation_table_region.fst
* Somatic copy number alterations: 20221109_TRACERx421_scna_table.fst

> Coordinates of mutations and CNAs in all listed files are in hg19.

## Required software and packages <a class="anchor" id="Required-software-and-packages"></a>

In order to load the data into R, perform minimal data wrangling and visualization, we reccomend using `dplyr`, `fst` and `ggplot2` packages freely avaible from CRAN. But please do feel free to use any packages you are used to or find helpful!

In [31]:
suppressWarnings(suppressPackageStartupMessages(library(dplyr)))
suppressWarnings(suppressPackageStartupMessages(library(fst)))
suppressWarnings(suppressPackageStartupMessages(library(ggplot2)))

## I am not able to install required software/packages on my computer, what should I do? <a class="anchor" id="I-am-not-able-to-install"></a>

Insert here how to use google colab

## Somatic mutation data <a class="anchor" id="Somatic-mutation-data"></a>

### Reading data into R <a class="anchor" id="Somatic-Reading"></a>

The code below shows how to load mutational data in the memory of your computer and to simplify some columns for your convinience. 
> **Please do use the snippet below to read your data in as we performed some modifications in order to streamline your future analysis**

In [11]:
# read table into memory
mut_table = read_fst('tracerx_data/figurecode/data/20221123_TRACERx421_mutation_table_region.fst')

# simplify RegionID, please do not skip this step
mut_table$is_lymphnode = grepl('LN', mut_table$RegionID)
mut_table$RegionID = gsub('.*[.]R|.*LN', '', mut_table$RegionID)
mut_table$RegionID = gsub('01', '1', mut_table$RegionID)
lymph_ids = mut_table$is_lymphnode == T
mut_table$RegionID[lymph_ids] = paste0('LN', mut_table$RegionID[lymph_ids])
tumor_ids = mut_table$is_lymphnode == F
mut_table$RegionID[tumor_ids] = paste0('R', mut_table$RegionID[tumor_ids])
mut_table$RegionID = apply(mut_table, 1, 
                           function(x) paste0(x['tumour_id'], ':', 
                                              x['RegionID']))
rm(tumor_ids, lymph_ids)

                           
# simplify mutation_id, please do not skip this step
muts_to_simplify <- grepl(';', mut_table$mutation_id)
mut_table[muts_to_simplify, ]$mutation_id = sapply(mut_table[muts_to_simplify, ]$mutation_id,
                                                   function(x) sub(':', 
                                                                   paste0('_', 
                                                                          gsub('.*;', '', x),
                                                                          ':'),
                                                                   x))
mut_table[muts_to_simplify, ]$mutation_id = gsub(';.*', '',
                                                 mut_table[muts_to_simplify, ]$mutation_id)

# simplify MajorCPN_SC and MinorCPN_SC, please do not skip this step
mut_table$r_id = gsub('.*:', '', mut_table$RegionID)
mut_table$r_id = paste0('.*', mut_table$r_id, ':')
mut_table$MajorCPN_SC = apply(mut_table, 1, 
                              function(x) gsub(x['r_id'], '', x['MajorCPN_SC']))
mut_table$MajorCPN_SC = gsub(';.*', '', mut_table$MajorCPN_SC)
mut_table$MajorCPN_SC = suppressWarnings(as.integer(mut_table$MajorCPN_SC))
mut_table$MinorCPN_SC = apply(mut_table, 1, 
                              function(x) gsub(x['r_id'], '', x['MinorCPN_SC']))
mut_table$MinorCPN_SC = gsub(';.*', '', mut_table$MinorCPN_SC)
mut_table$MinorCPN_SC = suppressWarnings(as.integer(mut_table$MinorCPN_SC))                                                 
                                                    
# re-order columns for clarity
mut_table = mut_table[, c('tumour_id', 'RegionID', 'is_lymphnode', 'mutation_id', 
                          'var_count', 'depth', 'Is.present.region', 
                          'CCF_pyclone_SC', 'CCF_phylo_SC', 'mut_cpn_SC', 
                          'MajorCPN_SC', 'MinorCPN_SC', 'PyCloneCluster_SC',
                          'PyCloneClonal_SC', 'cleanCluster_SC')]

In case your computer can't handle such big data frame, you can download the pre-processed version [here]().

### Column description <a class="anchor" id="Somatic-Columns"></a>

In [12]:
head(mut_table)

,tumour_id,RegionID,is_lymphnode,mutation_id,var_count,depth,Is.present.region,CCF_pyclone_SC,CCF_phylo_SC,mut_cpn_SC,MajorCPN_SC,MinorCPN_SC,PyCloneCluster_SC,PyCloneClonal_SC,cleanCluster_SC
,<chr>,<chr>,<lgl>,<chr>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<chr>,<lgl>
1,CRUK0005,CRUK0005:R1,FALSE,CRUK0005:19:34291428:G,5,159,TRUE,0.54,0.54,0.54,4,2,3,S,TRUE
2,CRUK0005,CRUK0005:R1,FALSE,CRUK0005:19:9084770:C,149,498,TRUE,1.16,1.16,4.57,4,0,2,C,TRUE
3,CRUK0005,CRUK0005:R1,FALSE,CRUK0005:1:2160418:G,27,171,TRUE,0.96,0.96,2.37,2,1,2,C,TRUE
4,CRUK0005,CRUK0005:R1,FALSE,CRUK0005:19:40095981:G,47,507,TRUE,1.21,1.21,1.25,2,0,2,C,TRUE
5,CRUK0005,CRUK0005:R1,FALSE,CRUK0005:19:41313703:C,0,616,FALSE,0.00,0.00,0.00,2,0,5,S,TRUE
6,CRUK0005,CRUK0005:R1,FALSE,CRUK0005:20:31387110:T,68,389,TRUE,0.92,0.92,2.76,3,1,2,C,TRUE


* `tumour_id` - unique tumor identifier
* `RegionID` - a unique identifier for a region inside the tumor. As it is a multiregional data, one tumor can have multiple `RegionID` linked to it. An identifier has the following format: tumour_id:region name. For example, CRUK0005:R1 is an identifier for the region 1 extracted from tumor CRUK0005 during the surgery. If region name starts with **"R"** it means that it was extracted from the actual tumor, and if it starts with **"LN"** the extraction was performed from the lymph node.
* `is_lymphnode` - boolean, indicating if extraction was performed from the actual tumor mass (FALSE) or from a lymph node (TRUE).
* `mutation_id` - a unique identifier for detected somatic mutation. An identifier has the following format: tumour_id:chromosome:position:alternative base pair.
* `var_count` - number of reads covering that genomic position with the alternative base (mutated base pair) detected at the position.
* `depth` - total depth of sequencing (number of reads) at this position.
* `Is.present.region` - boolean indicator (TRUE/FALSE) of mutation's presence in this region of a tumor.
* `CCF_pyclone_SC` - cancer cell fraction (CCF) of a mutation derived from pyclone. CCF is a proportion of cancer cells that contain this mutation in the region, where 100 means that all cancer cells from the extracted region contain that mutation.
* `CCF_phylo_SC` - cancer cell fraction (CCF) of a mutation derived from a phylogenetic tree.
* `mut_cpn_SC ` - mutation copy number
* `MajorCPN_SC` - copy number of the major allele
* `MinorCPN_SC` - copy number of the minor allele
* `PyCloneCluster_SC` - cluster id from pyclone to which mutation belongs
* `PyCloneClonal_SC ` - clonality status of a mutation: clonal (C) or subclonal(S)
* `cleanCluster_SC` - cluster id from pyclone to which mutation belongs, inconsistent clusters were removed

### How to extract mutation coordinates <a class="anchor" id="Somatic-Coordinates"></a>

It may be useful for some of your analysis to have a separate data frame with the coordinates of all detected somatic mutations. The snippet below provides an example of how the coordinates can be derived from mutation ids.



In [32]:
mutation_coords = strsplit(mut_table$mutation_id, ":")
mutation_coords = as.data.frame(do.call(rbind, mutation_coords))
colnames(mutation_coords) = c('tumour_id', 'chr', 'pos', 'ALT')
head(mutation_coords)

,tumour_id,chr,pos,ALT
,<chr>,<chr>,<chr>,<chr>
1,CRUK0005,19,34291428,G
2,CRUK0005,19,9084770,C
3,CRUK0005,1,2160418,G
4,CRUK0005,19,40095981,G
5,CRUK0005,19,41313703,C
6,CRUK0005,20,31387110,T


> IMPORTANT NOTES

* Plot overview of number of regions per tumor - or see the figure X
* Plot overview of number of mutations per tumor and per region
* Distributions of depth and var count

## SCNA table
TRACERx cohort consists of 421 patients, from which a total of 1644 tumor regions were sampled and subjected to whole-exome sequencing. Out of all these samples, there were only 401 tumors for which it was possible to reconstruct phylogenetic trees (only tumors where the purity was sufficient to determine copy number states in at least two regions were retained), for a total of 1428 regions.

This table contains the information about SCNA detected in each tumor sample corresponding to a specific region, inferred from *WES* data. 

In [ ]:
cna_table = read_fst('tracerx_data/figurecode/data/20221109_TRACERx421_scna_table.fst')
head(cna_table)

The columns are the following:
* `patient_id` : patient identifier
* `tumour_id` : tumor identifier
* `sample` Sample identfier. Example: CRUK0000_SU_T1-R1 stands for the sample of patient identifier CRUK0000 that was taken from region 1 (R1) of tumour 1 (T1), which was resected during primary surgery (SU).
* `chr` - `startpos` - `endpos`: coordinates of the copy number segment.
* `n.het` : number of heterozygous SNPs called on the segment.
* `cnTotal` : segment total copy number (ASCAT)
* `nMajor` : copy number of the major allele (ASCAT) (?)
* `nMinor` : copy number of the minor allele (ASCAT) (?)
* `Ploidy` : ploidy value of the sample (ASCAT) 
* `ACF` : Aberrant Cell Fraction (fraction of tumor cells in the sample) (ASCAT)
* `nAraw` : raw Allele specific copy number for the A-allele (ASCAT)
* `nBraw` : raw Allele specific copy number for the B-allele (ASCAT)
* `cpn_event_vs_ploidy` : copy number status with respect to the sample ploidy. It is a string whose value can be one of `c('neutral', 'gain', 'amp', 'loss', 'deep_loss')` E.g., if ploidy is 3 and cnTotal is 3, `cpn_event_vs_ploidy` will be `'neutral'`. If ploidy is 3 and cnTotal is 4, then `cpn_event_vs_ploidy` will be `'gain'`.
* `MSAI` : Mirrored Subclonal Allelic Imbalance. Logical indicating whether the corresponding allele specific copy number event corresponds to a mirrored subclonal allelic imbalance.
* `samp_used_to_phase` : ID of the sample whose allelic imbalance data was used to phase SNPs and estimate haplotype specific copy number profiles. ?: was this the result of the re-run with Refphase?

## Pathology and clinical data
TRACERx cohort consists of 421 patients, harbouring 432 genomically independent tumors. Each line in this dataframe corresponds to one of these tumours, reporting clinical data and histological features.

In [ ]:
tumor_df = readRDS('tracerx_data/figurecode/data/20221109_TRACERx421_all_tumour_df.rds')
head(tumor_df)

The columns in the dataframe are the following:
* `tumour_id_muttable_cruk` : ID of the tumor sample
* `cruk_id` : ID of the tumor sample (same information as the previous column)
* `clinical_sex`: patient sex
* `age` : patient age
* `ethnicity` : patient ethnicity. It is one of: `c('White- Irish', 'White- British', 'Caribbean', 'Black', 'White and Asian', 'White-European', 'White- Other', 'South American','Indian', 'Middle eastern', 'White and Black')`


In [ ]:
ggplot(data = tumor_df,
    mapping = aes(x = '', fill = ethnicity)) +
    geom_bar() + coord_polar("y", start=0) + theme(axis.text = element_blank()) 

* `Histology_per_tumour_id_muttable` : full name of the tumor sample histological subtype. It's one of: `c('Invasive adenocarcinoma', 'LCNEC', 'Squamous cell carcinoma', 'Pleomorphic carcinoma', 'Adenosquamous carcinoma', 'Carcinosarcoma', 'combined LUAD and LCNEC', 'Large cell carcinoma', 'Collision LUAD and LUSC')`.  There are 248 Invasive Adenocarcinomas, 138 Squamous cell carcinomas and 46 other histological subtypes. The labels are distributed as follows:

In [ ]:
ggplot(data = tumor_df,
    mapping = aes(x = Histology_per_tumour_id_muttable , fill = Histology_per_tumour_id_muttable)) +
    geom_bar() + # coord_polar("y", start=0) + 
	theme(legend.position = 'none', 
		axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 17), 
		text = element_text(size = 15)) 

In [ ]:
tumor_df %>% select(pN_stage_per_lesion, pT_stage_per_lesion) %>% unique

* `histology_3` : for samples that are either `'Invasive adenocarcinoma'` or `'Squamous cell carcinoma'` this column is equal to LUAD and LUSC respectively. For all other histologies, this column contains 'Other'.
* `LUAD_pred_subtype_with.IMA_per_tumour`. For LUAD samples, this column contains the tissue architecture growth pattern. Such patterns are inferred from diagnostic H&E slides, and they can be divided in two categories: low-grade (lepidic, mid grade, papillary and acinar) and high-grade (cribriform, micropapillary and solid).
* `site_per_lesion` : location of the tumor. It is one of `c('Right Upper Lobe', 'Left Upper Lobe', 'Left Lower Lobe', 'Right Lower Lobe', 'Right Middle Lobe', 'Left Upper lobe')`
* `pT_stage_per_lesion` : (?) I don't know what this stands for.
* `pN_stage_per_lesion` : (?) I don't know what this stands for. The values are 0, 1 and 2
* `size_pathology_per_lesion` : pathological tumor size. (?) how is it calculated? It ranges between 7 and 140
* `vascular_invasion_per_lesion` : logical indicating the presence of lymphovascular invasion.
* `pleural_invasion_per_lesion` : logical indicating the presence of visceral pleural invasion.
* `smoking_status_merged` : smking status of the patient. Cna be `c('Ex-Smoker' 'Smoker' and 'Never Smoked')`
* `cigs_perday` : number of cigarettes smoked per day. There are non integer values because some patients smoke cigars or pipes, whose amoiunt was converted to the equivalent in cigarettes counts. One cigar is assumed to be equivalent to approximately 1.5 cigarettes, and for pipes, one bowl of tobacco is equivalent to 2.5 cigarettes, following the table presented [here](http://www.smoking2.nes.scot.nhs.uk/module4/working-out-cigarette-equivalents.html) for conversions.
* `years_smoking` : duration fof smoking in terms of years.
* `pack_years` : number of cigarettes packs smoked per year.
* `is.family.lung` : (?) I don't know what this stands for. Probably is a logical indicating whether there is a history of lung cancer in the family.
* `pathologyTNM` : pathological Tumor, Node and Metastasis (TNM) stage (? check I defined it right)


In [ ]:
sort(unique(tumor_df$size_pathology_per_lesion))

In [ ]:
sort(unique(tumor_df$vascular_invasion_per_lesion))

In [ ]:
unique(tumor_df$cigs_perday)